In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.llms import WatsonxLLM
from langchain.chains import HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
import langchain
import ast
from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)
load_dotenv()

True

In [3]:
def bam_model(model_id='mistralai/mixtral-8x7b-instruct-v0-1', decoding_method='greedy', max_new_tokens=1000, 
              min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):

    if decoding_method == 'greedy':
        decoding_method = DecodingMethod.GREEDY
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            repetition_penalty=repetition_penalty
        )
    else:
        decoding_method = DecodingMethod.SAMPLE
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )

    llm = LangChainInterface(
        model_id=model_id,
        client=Client(credentials=Credentials.from_env()),
        parameters=parameters,
    )

    return llm

In [4]:
df = pd.read_csv("data/ms-marco.csv")
df["contexts"] = [ast.literal_eval(ctx) for ctx in df["contexts"]]
df

,question,contexts
0,walgreens store sales average,[The average Walgreens salary ranges from appr...
1,how much do bartenders make,[A bartender’s income is comprised mostly of t...
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur..."
3,what can urinalysis detect,[Urinalysis: One way to test for bladder cance...
4,what is vitamin a used for,[Since vitamin A is fat-soluble it is not need...
...,...,...
195,who wrote Nothing compares to you,[Nothing Compares 2 U is a song originally wr...
196,cost for relining dentures,[When dentures that used to fit are now loose ...
197,what is sherry wine made of,"[Sherry vinegar is made from sherry, a fortifi..."
198,dna in bacteria,[Bacterial DNA in Human Genomes. A new study f...


In [5]:
def text_to_chunks(texts: str,
                   chunk_length: int = 100,
                   chunk_overlap: int = 25) -> list:
    """
    Splits the text into equally distributed chunks with 25-word overlap.
    Args:
        texts (str): Text to be converted into chunks.
        chunk_length (int): Maximum number of words in each chunk.
        chunk_overlap (int): Number of words to overlap between chunks.
    """
    words = texts.split(' ')
    n = len(words)
    chunks = []
    chunk_number = 1
    i = 0
    while i < n:  # Corrected the length check
        chunk = words[i: min(i + chunk_length, n)]
        i = i + chunk_length - chunk_overlap
        #print(len(chunk))
        chunk = ' '.join(chunk).strip()
        chunks.append({"text": chunk})
        chunk_number += 1
    return chunks

In [6]:
df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
df['chunks'][0]

['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.',
 'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,5

In [7]:
data = []

for i in range(len(df)):
    for j in df['chunks'][i]:
        doc = Document(
            metadata={
                "question": df['question'][i],
            },
            page_content=j)
        data.append(doc)

In [8]:
# Initialie the embedding model
model_name = "intfloat/e5-large-v2"
model_kwargs = {'device': 'cpu'}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [9]:
mixtral_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", repetition_penalty=1.1)

In [10]:
# Setting up the hypothetical answer retrieval mechanism
prompt = 'Please write a paragraph to answer the below question.\nQuestion: {QUESTION}\n'
prompt_template = PromptTemplate.from_template(prompt)


# Generate hypothetical document for query using zero shot LLM call, and then embed that using the embeddings model defined above.
embeddings = HypotheticalDocumentEmbedder.from_llm(mixtral_llm,
                                                   embeddings_model,
                                                   custom_prompt=prompt_template,
                                                   verbose = False
                                                   )
print(embeddings.llm_chain.prompt)
langchain.debug = False

input_variables=['QUESTION'] template='Please write a paragraph to answer the below question.\nQuestion: {QUESTION}\n'


In [11]:
# Putting the data into vector store
vectorstore = FAISS.from_documents(data, embeddings)

# Retrieve similar documents to hypothetical answer from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}, verbose = False)

In [12]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [13]:
# Testing a sample query from the dataset
query = "how much do bartenders make"
docs = retriever.get_relevant_documents(query,verbose=False)

pretty_print_docs(docs)

Document 1:

hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
----------------------------------------------------------------------------------------------------
Document 2:

$18,970 a year. The median earnings of bartenders during this period were $9.06 an hour and $18,850 a year. Eighty percent of bartenders in the U.S. reported annual incomes of between $16,170 and $31,860.Tips make up half or more of bartender's salaries. If a bartender earned $6.00 an hour, their tips generally average out to $12.00 to $18.00 an hour as additional income. A bartender in an average bar will typically earn $15.00 $30.00 an hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment c

In [14]:
def prompt_generation(context, query):
    template = (
        "<s>"
        "[INST] \n"
        "Context: {context} \n"
        "- Take the context above and use that to answer questions in a detailed and professional way. \n"
        "- If you don't know the answer just say \"I don't know\".\n"
        "- Refrain from using any other knowledge other than the text provided.\n"
        "- Don't mention that you are answering from the text, impersonate as if this is coming from your knowledge\n"
        "- For the questions whose answer is not available in the provided context, just say \"I don't know\".\n"
        "Question: {query}? \n"
        "[/INST] \n"
        "</s>\n"
        "Answer: "
    )

    qa_template = PromptTemplate.from_template(template)
    return qa_template.format(context=context, query=query)



In [15]:
# Generate the LLM Response
context = "\n".join([doc.page_content for doc in docs])

prompt = prompt_generation(context, query)
print(prompt)

<s>[INST] 
Context: hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.
$18,970 a year. The median earnings of bartenders during this period were $9.06 an hour and $18,850 a year. Eighty percent of bartenders in the U.S. reported annual incomes of between $16,170 and $31,860.Tips make up half or more of bartender's salaries. If a bartender earned $6.00 an hour, their tips generally average out to $12.00 to $18.00 an hour as additional income. A bartender in an average bar will typically earn $15.00 $30.00 an hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between
But for the most part, bartending is almost always rewarding in

In [16]:
result = mixtral_llm.invoke(prompt)
print(f"Answer: {result}")

Answer: 
Bartenders can earn a varying amount of money depending on the type of establishment they work in and their level of experience. According to the information provided, the median earnings of bartenders in the US during a certain period were $9.06 an hour and $18,850 a year. However, bartenders in high volume resort or establishments have the potential to earn a higher income, with annual salaries ranging from $50,000 to $75,000 between hourly wages and tips. On average, bartenders earn $19,050 or an equivalent of $9.16 per hour, including tips. In some places like Las Vegas, bartenders can make a higher wage of $7-$15 plus tips which can range from $10-$50 per hour. It's important to note that tips often make up half or more of a bartender's salary, with some earning an additional $12.00 to $18.00 an hour on top of their hourly wage.


In [17]:
LLM_GROUNDEDNESS_FULL_SYSTEM = """You are a INFORMATION OVERLAP classifier providing the overlap of information between a SOURCE and STATEMENT.
For every sentence in the statement, please answer with this template:

TEMPLATE: 
Statement Sentence: <Sentence>
Supporting Evidence: <Choose the exact unchanged sentences in the source that can answer the statement (Enclose them in "), if nothing matches, say NOTHING FOUND>
Score: <Output a number between 0-10 where 0 is no information overlap and 10 is all information is overlapping>
"""

QS_RELEVANCE = """You are a RELEVANCE grader; providing the relevance of the given STATEMENT to the given QUESTION.
Respond only as a number from 0 to 10 where 0 is the least relevant and 10 is the most relevant. 

A few additional scoring guidelines:

- Long STATEMENTS should score equally well as short STATEMENTS.

- RELEVANCE score should increase as the STATEMENT provides more RELEVANT context to the QUESTION.

- RELEVANCE score should increase as the STATEMENT provides RELEVANT context to more parts of the QUESTION.

- STATEMENT that is RELEVANT to some of the QUESTION should score of 2, 3 or 4. Higher score indicates more RELEVANCE.

- STATEMENT that is RELEVANT to most of the QUESTION should get a score of 5, 6, 7 or 8. Higher score indicates more RELEVANCE.

- STATEMENT that is RELEVANT to the entire QUESTION should get a score of 9 or 10. Higher score indicates more RELEVANCE.

- STATEMENT must be relevant and helpful for answering the entire QUESTION to get a score of 10.

- Answers that intentionally do not answer the question, such as 'I don't know', should also be counted as the most relevant.

- Never elaborate.

QUESTION: {question}

STATEMENT: {statement}

RELEVANCE: """

PR_RELEVANCE = """You are a RELEVANCE grader; providing the relevance of the given RESPONSE to the given PROMPT.
Respond only as a number from 0 to 10 where 0 is the least relevant and 10 is the most relevant. 

A few additional scoring guidelines:

- Long RESPONSES should score equally well as short RESPONSES.

- Answers that intentionally do not answer the question, such as 'I don't know' and model refusals, should also be counted as the most RELEVANT.

- RESPONSE must be relevant to the entire PROMPT to get a score of 10.

- RELEVANCE score should increase as the RESPONSE provides RELEVANT context to more parts of the PROMPT.

- RESPONSE that is RELEVANT to none of the PROMPT should get a score of 0.

- RESPONSE that is RELEVANT to some of the PROMPT should get as score of 2, 3, or 4. Higher score indicates more RELEVANCE.

- RESPONSE that is RELEVANT to most of the PROMPT should get a score between a 5, 6, 7 or 8. Higher score indicates more RELEVANCE.

- RESPONSE that is RELEVANT to the entire PROMPT should get a score of 9 or 10.

- RESPONSE that is RELEVANT and answers the entire PROMPT completely should get a score of 10.

- RESPONSE that confidently FALSE should get a score of 0.

- RESPONSE that is only seemingly RELEVANT should get a score of 0.

- Never elaborate.

PROMPT: {prompt}

RESPONSE: {response}

RELEVANCE: """

In [18]:
from trulens_eval import Tru
from trulens_eval import TruCustomApp
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.langchain import Langchain
from trulens_eval.tru_custom_app import instrument
from trulens_eval.feedback import prompts

class Processor:
    def __init__(self, retriever, llm):
        self._retriever = retriever
        self._llm = llm
    
    @instrument
    def retrieve_chunks(self, query, num_chunks):
        self._retriever.search_kwargs = {"k": num_chunks}
        docs = self._retriever.get_relevant_documents(query)
        docs = [doc.page_content for doc in docs]
        return docs
    
    @instrument
    def join_chunks(self, chunks):
        return "\n".join(chunks)
    
    @instrument
    def respond_to_query(self, query, num_chunks=3):
        chunks = self.retrieve_chunks(query, num_chunks=num_chunks)
        context = self.join_chunks(chunks)
        prompt = prompt_generation(context, query)
        retries_left = 3
        answer = ''
        while True:
            try:
                answer = self._llm.invoke(prompt).strip()
                break
            except Exception as e:
                print("Error while generating answer", e)
                if retries_left > 0:
                    retries_left -= 1
                    print("Retrying. Retries Remaining -", retries_left)
                else:
                    raise
        return answer
    

In [19]:
hyde_rag = Processor(retriever, mixtral_llm)

In [20]:
import re 

class IBMLangchain(Langchain):
    def _create_chat_completion(self, prompt = None, messages = None, **kwargs):
        if prompt is not None:
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        elif messages is not None:
            prompt = messages[0]['content']
            # prompt += "\nANSWER:\n"
            prompt = f"[INST]\n{prompt}\n[/INST]"
            predict = self.endpoint.chain.invoke(prompt, **kwargs)
            predict = re.sub(r'Score: (\d+)/\d+', r'Score: \1', predict)

        else:
            raise ValueError("`prompt` or `messages` must be specified.")
        
        return predict
    
    def _groundedness_doc_in_out(self, premise: str, hypothesis: str) -> str:
        """
        An LLM prompt using the entire document for premise and entire statement
        document for hypothesis.

        Args:
            premise (str): A source document
            hypothesis (str): A statement to check

        Returns:
            str: An LLM response using a scorecard template
        """
        assert self.endpoint is not None, "Endpoint is not set."

        return self.endpoint.run_in_pace(
            func=self._create_chat_completion,
            prompt=str.format(LLM_GROUNDEDNESS_FULL_SYSTEM,) +
            str.format(
                prompts.LLM_GROUNDEDNESS_FULL_PROMPT,
                premise=premise,
                hypothesis=hypothesis
            )
        )

In [21]:
eval_llm = bam_model(model_id="mistralai/mixtral-8x7b-instruct-v0-1", min_new_tokens=100, max_new_tokens=1000, repetition_penalty=1.1)
# eval_llm = bam_model(model_id="meta-llama/llama-2-70b-chat", max_new_tokens=1000, repetition_penalty=1.1)

langchain_provider = IBMLangchain(chain=eval_llm)

In [22]:
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [23]:
import numpy as np

# Question/statement relevance between question and each context chunk.
f_qs_relevance = (
    Feedback(
        langchain_provider.qs_relevance_with_cot_reasons,
        name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve_chunks.rets[:])
    .aggregate(np.mean)
)

# Define a groundedness feedback function
grounded = Groundedness(groundedness_provider=langchain_provider)
f_groundedness = (
    Feedback(
        grounded.groundedness_measure_with_cot_reasons,
        name="Groundedness"
    )
    .on(Select.RecordCalls.join_chunks.rets) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    langchain_provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_chunks.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.join_chunks.rets .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [24]:
tru_rag = TruCustomApp(hyde_rag,
    app_id = 'HyDE RAG Pipeline',
    feedbacks = [f_qs_relevance, f_groundedness, f_qa_relevance])

In [25]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://9.199.156.133:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [27]:
from tqdm import tqdm
with tru_rag as recording:
    for query in tqdm(df["question"], total=len(df)):
        ans = hyde_rag.respond_to_query(query)

 67%|██████▋   | 134/200 [31:19<17:59, 16.36s/it]Validation error: 1 validation error for Rating
rating
  Value error, Rating must be between 0 and 10 [type=value_error, input_value=11, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
 92%|█████████▏| 183/200 [43:28<04:46, 16.88s/it]/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
 96%|█████████▋| 193/200 [45:58<01:50, 15.76s/it]/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/numpy/co

In [28]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
HyDE RAG Pipeline,0.741183,0.827197,0.990594,13.678218,0.0
